Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
ID = ""

---

## Laboratory 2

### Statistical Inference EE 2102575:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GabbySuwichaya/Statistical-Learning-EE575/blob/master/LAB2/main.ipynb)

- [Task 1. Neural network quiz](#task-1-neural-network-quiz)
- [Task 2. Implementatoin](#task-2-static-nn-vs-bnn-implementations)

Scoring

- [3 scores] Task 1. Neural Network Quiz
 
- [17 scores] Static NN Implementations  ... 
    
    1. [4 scores, if the model is correct] The first 3-5 layers are linear layers  + 1 activation (you can choose any function)

    2. [4 scores, if the data loading is correct]  Choose 5-10 features as the inputs (also gives the reason why) and 1 output for the house price. 

    3. [1 scores, if the data spliting is correct]  Split the `house-price` datasets into training 80% and testing 20%. In the training part, divide 10% of the training data for validation. 

    4. [3+3 scores, if the learning is correct] Show the progress of the training and validation losses.  

    5. [2 scores, if the results are reasonable] Perform the evaluation using MSE, MAE, and R^2 
    
 After you finished, please let me know and I will mark the codes...  


- [40 scores Homeworks]  BNN Implementations and compare with Static NN Homework

    1. [15 scores] BNN Model  
        - Bayesian Linear Model 
        - Bayesian Linear Layer
        - Gaussian Mixture Prior + any other auxiliary functions

    2. [10 scores] Training 
        - ELBO loss is correct 
        - Sampling is correct

    3. [4 scores] validation after each training 
        - Use reasonable measure to guage the results.

    4. [3+3 scores, if the learning is correct] Show the progress of the training and validation losses for both models.  

    5. [10 scores, if the results are reasonable] Perform the evaluation using MSE, MAE, and R^2 

--------------------------------------------------

#### Installing packages

In [ ]:
# ! pip3 install torch torchvision torchaudio 
# ! pip install torchviz 
# ! pip install graphviz

## Task 1. Neural Network Quiz

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
 
from torchviz import make_dot

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__() 
        self.fc1 = nn.Linear(10, 1)  
        self.fc2 = nn.Sigmoid()  
 
    def forward(self, x):  
        x = self.fc1(x) 
        x = self.fc2(x)  
        return x
     
# Move the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
simple_nn = SimpleNN() 

-----------------------------------------------

Q1. What is the number of learnable parameters in `SimpleNN` ?  Check this by coding and explain why

In [ ]:
def count_parameters(model):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
num_params = count_parameters(simple_nn)
print(num_params)

Explain in the cell below ...

-------------------------------------------------------------------------

Q2. Can you try to visualize the computational graph of `SimpleNN`? 

Hint! You can use `make_dot` imported from `torchviz` package

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

---------------------------------------------------------------------

Q3. Explain how the weights and biases are updated mathematically by showing the gradient calculation (just like in the Lecture 10).  

----------------------------------------------------------------------------------------

## Task 2. Static NN vs BNN Implementations  

Q4. Implement the following architectures for static DNN called `MyNN`. [17 scores]

    1. [4 scores, if the model is correct] The first 3-5 layers are linear layers  + 1 activation (you can choose any function)

    2. [4 scores, if the data loading is correct]  Choose 5-10 features as the inputs (also gives the reason why) and 1 output for the house price. 

    3. [1 scores, if the data spliting is correct]  Split the `house-price` datasets into training 80% and testing 20%. In the training part, divide 10% of the training data for validation. 

    4. [3+3 scores, if the learning is correct] Show the progress of the training and validation losses.  

    5. [2 scores, if the results are reasonable] Perform the evaluation using MSE, MAE, and R^2 

-----------------------------------------------------------------------------------------------------------
 

Q5. Implement a similar BNN called `MyBNN` with the same settings, and compare the results with `MyNN`. ...  

[40 scores Homeworks]  BNN Implementations and compare with Static NN Homework

    1. [15 scores] BNN Model  
        - Bayesian Linear Model 
        - Bayesian Linear Layer
        - Gaussian Mixture Prior + any other auxiliary functions

    2. [10 scores] Training 
        - ELBO loss is correct 
        - Sampling is correct

    3. [4 scores] validation after each training 
        - Use reasonable measure to guage the results.

    4. [3+3 scores, if the learning is correct] Show the progress of the training and validation losses for both models.  

    5. [10 scores, if the results are reasonable] Perform the evaluation using MSE, MAE, and R^2 

#### Customized data loader 

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X_train_, y_train_, X_test_=None, y_test_=None, mode="Train"):
        self.training_data  = X_train_
        self.training_label = y_train_
        
        if mode == "Train":
            self.data  = X_train_
            self.label = y_train_
            self.length = X_train_.shape[0]

        elif  mode == "Test" or mode == "Valid":
            assert X_test_ is not None
            assert y_test_ is not None

            self.data   = X_test_
            self.label  = y_test_
            self.length = X_test_.shape[0]
 

        self.XScaler    = StandardScaler()
        self.training_data = self.XScaler.fit_transform(self.training_data).astype("float32")
        self.data          = self.XScaler.transform(self.data).astype("float32") 

        self.YScaler = StandardScaler()
        self.training_label = self.YScaler.fit_transform(self.training_label).astype("float32") 
        self.label          = self.YScaler.transform(self.label).astype("float32")

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        sample = self.data[idx, :] 
        label  = self.label[idx] 
        return sample, label

In [ ]:
foldername = "house-prices"

import zipfile 
with zipfile.ZipFile("house-prices-advanced-regression-techniques.zip","r") as zip_ref:
    zip_ref.extractall(foldername) 

Example usage

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

foldername = "house-prices"
df = pd.read_csv('%s/train.csv' % foldername)  

Feat_List = ['HouseStyle']
df[['HouseStyle']] = df.loc[:,['HouseStyle']].apply(LabelEncoder().fit_transform)
X = df.loc[:, Feat_List].values
Y = df['SalePrice'].values.reshape(-1,1)  


X_train_, X_test_, y_train_, y_test_ = train_test_split(X, Y, test_size=0.2, random_state= 0) 
TestingData  = CustomDataset(X_train_, y_train_, X_test_, y_test_, mode="Test")

--------------------------------------------

Your code starts from here:

```
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import StandardScaler
    import pandas as pd

    foldername = "house-prices"
    df = pd.read_csv('%s/train.csv' % foldername)  

    Feat_List = ['HouseStyle', 'LotArea']
    df[['HouseStyle']] = df.loc[:,['HouseStyle']].apply(LabelEncoder().fit_transform)
    X = df.loc[:, Feat_List].values
    Y = df['SalePrice'].values.reshape(-1,1)  

```